In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

### 日誌サンプルデータの読み込み

In [5]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [6]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [7]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、不要文字を削除した上で1単語（とその読み、原形、品詞）毎にリストに格納

In [8]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

def mecabed_diary_list(diary_normalization):
    mecabed_diary = neologd_tagger.parse(diary_normalization)
    mecabed_diary_list = mecabed_diary.split('\n')
    mecabed_diary_list2 = mecabed_diary_list[0:-2] # EOSと’’(空白単語)を除外
    
## 各単語、用語を\tとカンマ「,」で区切り、原形のみを抽出して辞書型にして初期化したリストに格納するfor文をこの後に追加
# mecabed_diary_list2は['社長\tシャチョウ\t社長\t名詞-一般\t\t',  'が\tガ\tが\t助詞-格助詞-一般\t\t',・・・]というリストの形式になっている
# これについて、mecabed_diary_list2の
# 1番目の要素を['社長', 'シャチョウ', '社長', '名詞-一般', '', ''] 
# 2番目の要素を['が', 'ガ', 'が', '助詞-格助詞-一般', '', '']
# というような1単語、1用語毎に区切られたリストの形式にしたい

# re.split()は第一引数に正規表現パターン、第二引数に対象の文字列を指定
# 複数の異なる区切り文字（文字列）で分割させたい場合に使う
# パターンを「|」で区切るといずれかのパターンにマッチする
# 各パターンには正規表現の特殊文字を使うことももちろん可能だが、文字列をそのまま指定することも可能
# 参考：https://note.nkmk.me/python-split-rsplit-splitlines-re/
    diclist = []
    for word in mecabed_diary_list2:
        l = re.split('\t|,',word) # 各単語、用語は\tとカンマ「,」で区切られてるのでこれらの文字で区切る
        d = {'BaseForm':l[2]} # リストに{BaseForm:xx}といった形式で格納したいためリストの中身を辞書型{}にする
        diclist.append(d)
    return diclist

### 感情極性辞書データの読み込み、 各単語とそのスコアを辞書型で格納

In [9]:
PN_df = pd.read_csv('posi_nega_dic.csv')
word_list = list(PN_df['word']) # list型
score_list = list(PN_df['score']) #list型
PN_dict = dict(zip(word_list, score_list)) #list型とlist型をdict(zip())関数に適用させるとdictionary型に

### 日誌サンプルを形態素解析した単語と感情極性辞書の単語のスコアを紐づける

In [10]:
def get_word_score_dic(diclist):
    diclist_new = []

    for word in diclist:
        base = word['BaseForm']  # list型のdiclistの要素1つ1つは辞書型なのでword['key']という書き方でkeyに紐づくvalueを抽出できる
        if base in PN_dict:
            weight = float(PN_dict[base])  
        else:
            weight = 'NAN'             # その語がPN_dict(感性極性辞書)になかった場合
        word['Weight'] = weight
        diclist_new.append(word)
    return diclist_new

### 感情極性辞書に含まれる単語のみ残す

In [11]:
def delete_word_dic(diclist_new):
    dictlist_add_weight_list =[]
    
    # df['xx']=='xx' の否定は  df['xx']!='xx'
    for word in diclist_new:
        if word['Weight'] != 'NAN' :   # wordは辞書型なのでdiclist_new['Weight'] とkeyを指定したときのvalueを抽出できる書き方ができる、便利
            dictlist_add_weight_list.append(word)
    return dictlist_add_weight_list

### 単語のスコアを抽出

In [12]:
def get_weight(dictlist_add_weight_list):
    weight_list = []

    for score in dictlist_add_weight_list:
        weight = float(score['Weight'])
        weight_list.append(weight)
    return weight_list

### 単語のスコアの平均を算出

In [13]:
def get_mean(weight_list):
    #global weight_mean #weight_mean = str(0) とすれば記述する必要なし
    if len(dictlist_add_weight_list) > 0:
        weight_mean = np.mean(weight_list)
    else:
        # weight_mean =='NA'
        weight_mean = str(0) # weight_mean =='NA'だと1つ下の関数実行部分で「'weight_mean' referenced before assignment」とエラー
    return weight_mean

### 日誌データのテキスト１つずつ関数を実行

In [14]:
score_list= []

for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)    
    diclist = mecabed_diary_list(diary_normalization)    
    diclist_new = get_word_score_dic(diclist)    
    dictlist_add_weight_list = delete_word_dic(diclist_new)    
    weight_list = dictlist_add_weight_list = get_weight(dictlist_add_weight_list)    
    score = get_mean(weight_list)
    score_list.append(score)

In [15]:
for score in score_list:
    print(score)

-0.46554470000000003
-0.27814381818181827
-0.25576150000000003
-0.31583062
-0.1432513333333333
-0.6789305
-0.48560359999999997


In [16]:
output = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [17]:
output['score'] = score_list

In [18]:
output

,text,score
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...,-0.465545
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...,-0.278144
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。,-0.255762
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。,-0.315831
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。,-0.143251
5,2018年4月から2018年6月までの上場会社数は20社。,-0.678930
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。,-0.485604


In [21]:
# csvファイルで出力
# 出力の際、csvファイル読み込み時と同じ encoding='utf-8'にすると日本語文字化けが発生
# encoding='shift-jis'で日本語文字化けが解消
# 自動でindexがついてきたのでそれを削除
output.to_csv('SBI_Financial statement_201903_add_score.csv', encoding='shift-jis', index=False)

##### 参考URL:https://www.statsbeginner.net/entry/2017/05/07/091435